In [1]:
!git clone https://github.com/snehahs03/medical-image-analysis.git

Cloning into 'medical-image-analysis'...
remote: Enumerating objects: 3757, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 3757 (delta 8), reused 30 (delta 4), pack-reused 3723
Receiving objects: 100% (3757/3757), 336.79 MiB | 42.00 MiB/s, done.
Resolving deltas: 100% (8/8), done.
Checking out files: 100% (3887/3887), done.


In [2]:
import os
from glob import glob

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding

from keras.preprocessing.image import ImageDataGenerator
from keras import layers
from keras import Model
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from keras.applications.densenet import DenseNet201
%matplotlib inline
import matplotlib.pyplot as plt

import json
import math
import cv2
from PIL import Image
from keras.applications import ResNet50,MobileNet, DenseNet201, InceptionV3, NASNetLarge, InceptionResNetV2, NASNetMobile
from keras.callbacks import Callback, ModelCheckpoint, ReduceLROnPlateau, TensorBoard
from keras.models import Sequential
from sklearn.metrics import cohen_kappa_score, accuracy_score
import scipy
from tqdm import tqdm
import tensorflow as tf
from keras import backend as K
import gc
from functools import partial
from sklearn import metrics
from collections import Counter
import itertools

In [4]:
def Dataset_loader(DIR, RESIZE, sigmaX=10):
    IMG = []
    read = lambda imname: np.asarray(Image.open(imname).convert("RGB"))
    for IMAGE_NAME in tqdm(os.listdir(DIR)):
        PATH = os.path.join(DIR,IMAGE_NAME)
        _, ftype = os.path.splitext(PATH)
        if ftype == ".jpg":
            img = read(PATH)

            img = cv2.resize(img, (RESIZE,RESIZE))

            IMG.append(np.array(img))
    return IMG

eczema_train = np.array(Dataset_loader('/content/medical-image-analysis/train/Eczema Photos', 224))
melanoma_train = np.array(Dataset_loader('/content/medical-image-analysis/train/Melanoma Skin Cancer Nevi and Moles',224))
psoriasis_train = np.array(Dataset_loader('/content/medical-image-analysis/train/Psoriasis pictures Lichen Planus and related diseases',224))

#labelling 
eczema_train_label = np.zeros(len(eczema_train))
melonoma_train_label = np.zeros(len(melanoma_train))
psoriasis_train_label = np.zeros(len(psoriasis_train))


X_train = np.concatenate((eczema_train, melanoma_train, psoriasis_train), axis=0)
y_train = np.concatenate((eczema_train_label, melonoma_train_label, psoriasis_train_label), axis=0)

s = np.arange(X_train.shape[0])
np.random.shuffle(s)
X_train = X_train[s]
y_train = y_train[s]
 

#train and evaluation split
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, 
    test_size=0.2, 
    random_state=11
)

100%|██████████| 1405/1405 [00:09<00:00, 152.95it/s]


In [5]:
X_train.shape, X_val.shape

((2482, 224, 224, 3), (621, 224, 224, 3))

In [6]:
y_train.shape, y_val.shape

((2482,), (621,))

In [7]:
y_train = to_categorical(y_train)
y_val = to_categorical(y_val)

In [8]:
y_train.shape, y_val.shape

((2482, 1), (621, 1))

In [9]:
pre_trained_model = InceptionResNetV2(input_shape=(192, 256, 3), include_top=False, weights="imagenet")

219062272/219055592 [==============================] - 2s 0us/step


In [10]:
for layer in pre_trained_model.layers:
    print(layer.name)
    if hasattr(layer, 'moving_mean') and hasattr(layer, 'moving_variance'):
        layer.trainable = True
        K.eval(K.update(layer.moving_mean, K.zeros_like(layer.moving_mean)))
        K.eval(K.update(layer.moving_variance, K.zeros_like(layer.moving_variance)))
    else:
        layer.trainable = False

print(len(pre_trained_model.layers))

input_1
conv2d
batch_normalization
activation
conv2d_1
batch_normalization_1
activation_1
conv2d_2
batch_normalization_2
activation_2
max_pooling2d
conv2d_3
batch_normalization_3
activation_3
conv2d_4
batch_normalization_4
activation_4
max_pooling2d_1
conv2d_8
batch_normalization_8
activation_8
conv2d_6
conv2d_9
batch_normalization_6
batch_normalization_9
activation_6
activation_9
average_pooling2d
conv2d_5
conv2d_7
conv2d_10
conv2d_11
batch_normalization_5
batch_normalization_7
batch_normalization_10
batch_normalization_11
activation_5
activation_7
activation_10
activation_11
mixed_5b
conv2d_15
batch_normalization_15
activation_15
conv2d_13
conv2d_16
batch_normalization_13
batch_normalization_16
activation_13
activation_16
conv2d_12
conv2d_14
conv2d_17
batch_normalization_12
batch_normalization_14
batch_normalization_17
activation_12
activation_14
activation_17
block35_1_mixed
block35_1_conv
block35_1
block35_1_ac
conv2d_21
batch_normalization_21
activation_21
conv2d_19
conv2d_22
batc

In [11]:
last_layer = pre_trained_model.get_layer('conv_7b_ac')
print('last layer output shape:', last_layer.output_shape)
last_output = last_layer.output

last layer output shape: (None, 4, 6, 1536)


In [12]:
# Flatten the output layer to 1 dimension
x = layers.GlobalMaxPooling2D()(last_output)
# Add a fully connected layer with 512 hidden units and ReLU activation
x = layers.Dense(512, activation='relu')(x)
# Add a dropout rate of 0.7
x = layers.Dropout(0.5)(x)
# Add a final sigmoid layer for classification
x = layers.Dense(7, activation='softmax')(x)

# Configure and compile the model

model = Model(pre_trained_model.input, x)
optimizer = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=True)
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

In [13]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 192, 256, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 95, 127, 32)  864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 95, 127, 32)  96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 95, 127, 32)  0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [14]:
train_datagen = ImageDataGenerator(rotation_range=60, width_shift_range=0.2, height_shift_range=0.2,
                                   shear_range=0.2, zoom_range=0.2, fill_mode='nearest')

train_datagen.fit(X_train)

val_datagen = ImageDataGenerator()
val_datagen.fit(X_val)

In [15]:
batch_size = 64
epochs = 3
history = model.fit_generator(train_datagen.flow(X_train,y_train, batch_size=batch_size),
                              epochs = epochs, validation_data = val_datagen.flow(X_val, y_val),
                              verbose = 1, steps_per_epoch=(X_train.shape[0] // batch_size), 
                              validation_steps=(X_val.shape[0] // batch_size))

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/3
38/38 [==============================] - 89s 1s/step - loss: 35.4390 - accuracy: 0.2184 - val_loss: 70295469686784.0000 - val_accuracy: 0.0000e+00
Epoch 2/3
38/38 [==============================] - 36s 933ms/step - loss: 102.1015 - accuracy: 0.1301 - val_loss: 1049730.5000 - val_accuracy: 0.0000e+00
Epoch 3/3
38/38 [==============================] - 36s 936ms/step - loss: 202.8722 - accuracy: 0.1082 - val_loss: 7545.6147 - val_accuracy: 0.0000e+00


In [16]:
pre_trained_model.layers[617].name

'mixed_7a'

In [17]:
for layer in pre_trained_model.layers[618:]:
    layer.trainable = True

In [18]:
optimizer = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['acc'])

In [19]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', patience=3, verbose=1, factor=0.5, 
                                            min_lr=0.000001, cooldown=2)

In [20]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 192, 256, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 95, 127, 32)  864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 95, 127, 32)  96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 95, 127, 32)  0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [21]:
batch_size = 64
epochs = 25
history = model.fit_generator(train_datagen.flow(X_train,y_train, batch_size=batch_size),
                              epochs = epochs, validation_data = val_datagen.flow(X_val, y_val),
                              verbose = 1, steps_per_epoch=(X_train.shape[0] // batch_size),
                              validation_steps=(X_val.shape[0] // batch_size),
                              callbacks=[learning_rate_reduction])

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/25
38/38 [==============================] - 53s 1s/step - loss: 450.8722 - acc: 0.0855 - val_loss: 298.1189 - val_acc: 0.0000e+00
Epoch 2/25
38/38 [==============================] - 37s 954ms/step - loss: 1273.4600 - acc: 0.0757 - val_loss: 349.7713 - val_acc: 0.0000e+00
Epoch 3/25
38/38 [==============================] - 37s 952ms/step - loss: 1700.8757 - acc: 0.0876 - val_loss: 399.6380 - val_acc: 0.0000e+00
Epoch 4/25
38/38 [==============================] - 37s 953ms/step - loss: 2097.5733 - acc: 0.0777 - val_loss: 423.3766 - val_acc: 0.0000e+00

Epoch 00004: ReduceLROnPlateau reducing learning rate to 4.999999873689376e-05.
Epoch 5/25
38/38 [==============================] - 37s 954ms/step - loss: 2409.9956 - acc: 0.0806 - val_loss: 521.5588 - val_acc: 0.0000e+00
Epoch 6/25
38/38 [==============================] - 37s 951ms/step - loss: 2648.2154 - acc: 0.0793 - val_loss: 602.7118 - val_acc: 0.0000e+00
Epoch 7/25
38/38 [==============================] - 37s 956ms/step - lo

In [22]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', patience=2, verbose=1, factor=0.5, 
                                            min_lr=0.000001, cooldown=1)

In [23]:
model.fit_generator(train_datagen.flow(X_train,y_train, batch_size=batch_size),
                     epochs = 5, validation_data = val_datagen.flow(X_val, y_val),
                     verbose = 1, steps_per_epoch=(X_train.shape[0] // batch_size),
                     validation_steps=(X_val.shape[0] // batch_size),
                     callbacks=[learning_rate_reduction])

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/5
38/38 [==============================] - 36s 929ms/step - loss: 3779.3247 - acc: 0.0691 - val_loss: 987.6337 - val_acc: 0.0000e+00
Epoch 2/5
38/38 [==============================] - 36s 934ms/step - loss: 3831.1870 - acc: 0.0728 - val_loss: 991.1956 - val_acc: 0.0000e+00
Epoch 3/5
38/38 [==============================] - 36s 940ms/step - loss: 3831.8611 - acc: 0.0778 - val_loss: 985.4648 - val_acc: 0.0000e+00

Epoch 00003: ReduceLROnPlateau reducing learning rate to 1e-06.
Epoch 4/5
38/38 [==============================] - 36s 939ms/step - loss: 3783.2646 - acc: 0.0707 - val_loss: 984.2931 - val_acc: 0.0000e+00
Epoch 5/5
38/38 [==============================] - 36s 946ms/step - loss: 3748.2026 - acc: 0.0885 - val_loss: 991.3665 - val_acc: 0.0000e+00
